In [12]:
from numpy.random import default_rng
import random
from neuron import h, gui
from neuron.units import ms, mV


rng = default_rng()

def poisson_process(start: float, max_t: float, avg_interval: float) -> list:
    t = start
    events = []
    while t < max_t:
        t += rng.exponential(avg_interval)
        events.append(t)
    # discard the last event, as it exceeds max_t
    if events:
        events = events[:-1]
    return events

def poisson_process_n(interval, n):
    interspikes = [random.expovariate(1. / interval) for _ in range(n)]
    t = 0
    times = []
    for interspike in interspikes:
        t += interspike
        times.append(t)
    return times

def poisson_process_duration(interval, duration):
    t = random.expovariate(1. / interval)
    times = []
    while t < duration:
        times.append(t)
        t += random.expovariate(1. / interval)
    return times

T_STOP = 100

e_stims = poisson_process(0, T_STOP, 5 * ms)
i_stims = poisson_process(0, T_STOP, 10 * ms)
testn = poisson_process_n(T_STOP, 10)
testsdu = poisson_process_duration(10*ms, T_STOP)


# example usage; assumes a cell object has been created; see below for add_custom_stimulus
# cell.add_custom_stimulus(e_stims, 0 * mV, 0.00025, 2 * ms)
# cell.add_custom_stimulus(i_stims, -80 * mV, 0.0005, 7.5 * ms)


In [14]:
def add_custom_stimulus(self, stim_times: list, reversal_potential: float, weight: float, tau: float) -> None:

    # add synapse
    syn = h.ExpSyn(self.soma(0.5))
    syn.tau = tau
    syn.e = reversal_potential * mV
    syn_current = h.Vector().record(syn._ref_i)
    self.syns.append(syn)
    self.syn_currents.append(syn_current)
    netstims = [h.NetStim() for _ in stim_times] 
    for netstim, stim_time in zip(netstims, stim_times):
        netstim.number = 1
        netstim.start = stim_time
        netcon = h.NetCon(netstim, syn)
        netcon.weight[0] = weight
        netcon.delay = 0 * ms
    self.netcons.append(netcon)
    self.stims.extend(netstims)

[16.464246677106782,
 36.339797438692266,
 42.1102040869965,
 44.97174822588503,
 50.93166294285691,
 56.48366709624702,
 57.542649601337025,
 90.93914864842064,
 94.68256660625313]